In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import datetime as dt
import time
import jinja2
from jinja2 import Environment, BaseLoader
from wrangler.aws import athena

In [4]:
sql_template = """
UNLOAD (
SELECT 
    u.machine_id,
    u.url_idc, 
    u.person_id, 
    u.ss2k, 
    u.time_id, 
    u.domain_name, 
    u.url_host, 
    u.url_dir, 
    u.url_page, 
    u.url_refer_domain, 
    u.url_refer_host, 
    u.url_refer_dir, 
    u.url_refer_page, 
    u.mimetype, 
    u.http_rc, 
    u.keywords, 
    u.html_title, 
    u.pattern_id
FROM "comscore"."url" as u
WHERE 
    u.year='{{year}}'
    and u.month='{{month}}'
    and u.day='{{day}}'
    and (u.domain_name LIKE '%twitch%')
) 
TO 's3://kdc-comscore/parquet-extracts/url_{{domain}}/year={{year}}/month={{month}}/day={{day}}' 
WITH ( format = 'parquet', compression = 'snappy' )
"""

template = Environment(loader=BaseLoader()).from_string(sql_template)

In [5]:
%%time

results = athena.run_daily(
    "kdc-admin",
    "us-east-2",
    template,
    start_date = dt.datetime(2019, 1, 1),
    end_date = dt.datetime(2019, 1, 2),
    database = "comscore",
    args = {
    },
    batch_size = 50
)
print(f"number of results: {len(results)}")

################################################################################
# executing for date range: ('2019', '01', '01') -> ('2019', '01', '02')
################################################################################
('2019', '01', '01'): STARTED
('2019', '01', '02'): STARTED
>>> Running: 1, Completed: 1
>>> Running: 0, Completed: 2
number of results: 2
CPU times: user 342 ms, sys: 43.7 ms, total: 386 ms
Wall time: 13.2 s


In [10]:
results[0]

(('2019', '01', '01'),
 {'QueryExecution': {'QueryExecutionId': '4dfee1f5-f09b-48a0-953b-0f0b0c3ed193',
   'Query': 'UNLOAD (\nSELECT \n    u.machine_id,\n    u.url_idc, \n    u.person_id, \n    u.ss2k, \n    u.time_id, \n    u.domain_name, \n    u.url_host, \n    u.url_dir, \n    u.url_page, \n    u.url_refer_domain, \n    u.url_refer_host, \n    u.url_refer_dir, \n    u.url_refer_page, \n    u.mimetype, \n    u.http_rc, \n    u.keywords, \n    u.html_title, \n    u.pattern_id\nFROM "comscore"."url" as u\nWHERE \n    u.year=\'2019\'\n    and u.month=\'01\'\n    and u.day=\'01\'\n    and (u.domain_name LIKE \'%twitch%\')\n) \nTO \'s3://kdc-comscore/parquet-extracts/url_/year=2019/month=01/day=01\' \nWITH ( format = \'parquet\', compression = \'snappy\' )',
   'StatementType': 'DML',
   'ResultConfiguration': {'OutputLocation': 's3://kdc-query-results/4dfee1f5-f09b-48a0-953b-0f0b0c3ed193'},
   'ResultReuseConfiguration': {'ResultReuseByAgeConfiguration': {'Enabled': False}},
   'QueryEx

In [18]:
rows = []
for r in results:
    date = "-".join(r[0])
    query_id = r[1]['QueryExecution']['QueryExecutionId']
    query_sql = r[1]['QueryExecution']['Query']
    state = r[1]['QueryExecution']['Status']['State']
    rows.append((date, query_id, query_sql, state))

pd.DataFrame(rows, columns=['date', 'query_id', "query_sql", "state"])

,date,query_id,query_sql,state
0,2019-01-01,4dfee1f5-f09b-48a0-953b-0f0b0c3ed193,"UNLOAD (\nSELECT \n u.machine_id,\n u.ur...",SUCCEEDED
1,2019-01-02,ae379792-38bb-48c6-9cdc-0a5949f08dda,"UNLOAD (\nSELECT \n u.machine_id,\n u.ur...",SUCCEEDED


In [ ]:
# ! aws s3 sync s3://kdc-comscore/parquet-extracts . --profile kdc-admin

In [31]:
df = pd.read_parquet("url_twitch_steam")

In [33]:
df.shape

(7009025, 21)